# Multi-GPU Training Example

Train a convolutional neural network on multiple GPU with TensorFlow.

This example is using TensorFlow layers, see 'convolutional_network_raw' example
for a raw TensorFlow implementation with variables.

- Author: Aymeric Damien
- Project: https://github.com/aymericdamien/TensorFlow-Examples/

## Training with multiple GPU cards

In this example, we are using data parallelism to split the training accross multiple GPUs. Each GPU has a full replica of the neural network model, and the weights (i.e. variables) are updated synchronously by waiting that each GPU process its batch of data.

First, each GPU process a distinct batch of data and compute the corresponding gradients, then, all gradients are accumulated in the CPU and averaged. The model weights are finally updated with the gradients averaged, and the new model weights are sent back to each GPU, to repeat the training process.

<img src="https://www.tensorflow.org/images/Parallelism.png" alt="Parallelism" style="width: 400px;"/>

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flatten and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

More info: http://yann.lecun.com/exdb/mnist/

In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
import time

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
num_workers = 2 # # of workers
num_gpus = 2
num_steps = 200
learning_rate = 0.001
batch_size = 1024
display_step = 10

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Build a convolutional neural network
def conv_net(x, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Convolution Layer with 64 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 64, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        x = tf.layers.max_pooling2d(x, 2, 2)

        # Convolution Layer with 256 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 256, 3, activation=tf.nn.relu)
        # Convolution Layer with 512 filters and a kernel size of 5
        x = tf.layers.conv2d(x, 512, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        x = tf.layers.max_pooling2d(x, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        x = tf.contrib.layers.flatten(x)

        # Fully connected layer (in contrib folder for now)
        x = tf.layers.dense(x, 2048)
        # Apply Dropout (if is_training is False, dropout is not applied)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)

        # Fully connected layer (in contrib folder for now)
        x = tf.layers.dense(x, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(x, n_classes)
        # Because 'softmax_cross_entropy_with_logits' loss already apply
        # softmax, we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out

In [3]:
# Build the function to average the gradients
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
cluster_mgpu = tf.train.ClusterSpec({
#     "ps": [
#         "10.138.0.4:3333",  # /job:ps/task:0
#         "10.138.0.4:3334",  # /job:ps/task:1
#     ],
    "worker": [
        "10.138.0.2:2221",  # /job:worker/task:0
        "10.138.0.4:2222",  # /job:worker/task:1
    ]}
)

# task_ps0 = tf.train.Server(cluster_mgpu, job_name="ps", task_index=0)
# task_ps1 = tf.train.Server(cluster_mgpu, job_name="ps", task_index=1)

task_worker0 = tf.train.Server(cluster_mgpu, job_name="worker", task_index=0)
task_worker1 = tf.train.Server(cluster_mgpu, job_name="worker", task_index=1)

In [5]:
#reset_graph()

# with tf.device(tf.train.replica_device_setter( # cluster=cluster_spec 처럼 클러스터 명세를 전달해도 ps_tasks 를 찾아서 이용함
#         ps_tasks=2,
#         ps_device="/job:ps",
#         worker_device="/job:worker")):
#     v1 = tf.Variable(1.0, name="v1")  # /job:ps/task:0 (defaults to /cpu:0) 에 할당
#     v2 = tf.Variable(2.0, name="v2")  # /job:ps/task:1 (defaults to /cpu:0) 에 할당
#     v3 = tf.Variable(3.0, name="v3")  # /job:ps/task:0 (defaults to /cpu:0) 에 할당
#     s = v1 + v2            # /job:worker (defaults to task:0/cpu:0) 에 할당
#     with tf.device("/task:1"):
#         p1 = 2 * s         # /job:worker/task:1 (defaults to /cpu:0) 에 할당
#         with tf.device("/cpu:0"):
#             p2 = 3 * s     # /job:worker/task:1/cpu:0 에 할당

# config = tf.ConfigProto()
# config.log_device_placement = True

# with tf.Session("grpc://10.138.0.4:2222", config=config) as sess:
#     v1.initializer.run()

In [5]:
import time
start = time.time()


with tf.device(tf.train.replica_device_setter( # cluster=cluster_spec 처럼 클러스터 명세를 전달해도 ps_tasks 를 찾아서 이용함
        #ps_tasks=2,
        #ps_device="/job:ps",
        worker_device="/job:worker")):
    tower_grads = []
    reuse_vars = False

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, num_input])
    Y = tf.placeholder(tf.float32, [None, num_classes])

    #X = tf.placeholder(tf.float16, [None, num_input])
    #Y = tf.placeholder(tf.float16, [None, num_classes])
    
    # Loop over all GPUs and construct their own computation graph
    for i in range(num_gpus):
        for j in range(num_workers):
            with tf.device('/task:%d/gpu:%d' % (j, i)):
                
                idx = i * num_gpus + j

                # Split data between GPUs
                _x = X[idx * batch_size: (idx+1) * batch_size]
                _y = Y[idx * batch_size: (idx+1) * batch_size]

                # Because Dropout have different behavior at training and prediction time, we
                # need to create 2 distinct computation graphs that share the same weights.

                # Create a graph for training
                logits_train = conv_net(_x, num_classes, dropout,
                                        reuse=reuse_vars, is_training=True)
                # Create another graph for testing that reuse the same weights
                logits_test = conv_net(_x, num_classes, dropout,
                                       reuse=True, is_training=False)

                # Define loss and optimizer (with train logits, for dropout to take effect)
                loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                    logits=logits_train, labels=_y))

                optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

                # https://stackoverflow.com/questions/42064941/tensorflow-float16-support-is-broken
                #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=1e-5) # for float16

                grads = optimizer.compute_gradients(loss_op)

                # Only first GPU compute accuracy
                if j == 0 and i == 0:
                    # Evaluate model (with test logits, for dropout to be disabled)
                    correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(_y, 1))
                    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

                reuse_vars = True
                tower_grads.append(grads)

    tower_grads = average_gradients(tower_grads)
    train_op = optimizer.apply_gradients(tower_grads)

    # Initializing the variables
    init = tf.global_variables_initializer()

    
config = tf.ConfigProto()
config.log_device_placement = True
with tf.Session("grpc://10.138.0.2:2221", config=config) as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    for step in range(1, num_steps + 1):
        # Get a batch for each GPU
        batch_x, batch_y = mnist.train.next_batch(batch_size * num_gpus * num_workers)
        # Run optimization op (backprop)
        ts = time.time()
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        te = time.time() - ts
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ": Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", %i Examples/sec" % int(len(batch_x)/te))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 1000 mnist test images
    print("Testing Accuracy:", \
        np.mean([sess.run(accuracy, feed_dict={X: mnist.test.images[i:i+batch_size],
        Y: mnist.test.labels[i:i+batch_size]}) for i in range(0, len(mnist.test.images), batch_size)]))
        
        
end = time.time()
print(end-start)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Step 1: Minibatch Loss= 2.0642, Training Accuracy= 0.454, 661 Examples/sec
Step 10: Minibatch Loss= 0.4398, Training Accuracy= 0.904, 6627 Examples/sec
Step 20: Minibatch Loss= 0.1351, Training Accuracy= 0.969, 6779 Examples/sec
Step 30: Minibatch Loss= 0.1104, Training Accuracy= 0.977, 6917 Examples/sec
Step 40: Minibatch Loss= 0.0621, Training Accuracy= 0.981, 6767 Examples/sec
Step 50: Minibatch Loss= 0.0475, Training Accuracy= 0.988, 6572 Examples/sec
Step 60: Minibatch Loss= 0.0269, Training Accuracy= 0.989, 6277 Examples/sec
Step 70: Minibatch Loss= 0.0346, Training Accuracy= 0.995, 6705 Examples/sec
Step 80: Minibatch Loss= 0.0259, Training Accuracy= 0.993, 6883 Examples/sec
Step 90: Minibatch Loss= 0.0418, Training Accuracy= 0.995, 6734 Examples/sec
Step 100: Minibatch Loss= 0.0193, Traini

single 2gpu 

Testing Accuracy: 0.9929468
64.05570006370544

2gpu * 2machine

Testing Accuracy: 0.99408084
173.84509682655334

2gpu * 2machine, no param server

Testing Accuracy: 0.994314
142.5129895210266

